In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Youssif/Legal data.csv")


In [ ]:
df.head()

,question,answer
0,على من ترفع الدعوى الجزائية الخاصة؟,ترفع دعوى الحق الخاص على المتهم إذا كان أهلًا،...
1,كم تبلغ المدة المحددة لإيقاف السير في الدعوى؟,يجوز وقف الدعوى بناء على اتفاق الخصوم على عدم ...
2,كم تبلغ رسوم إصدار ترخيص مكتب محاماة أجنبي؟,يبلغ رسم إصدار الترخيص لمكتب المحاماة الأجنبي ...
3,كم تبلغ رسوم تجديد ترخيص مكتب محاماة أجنبي؟,يكون رسم تجديد الترخيص لمكتب المحاماة الأجنبي ...
4,كم تبلغ قيمة التكاليف القضائية لطلب التنفيذ ال...,خمسمائة ريال.


In [ ]:
df.isna().sum()

,0
question,24
answer,93


In [ ]:
df.dropna(subset=["question", "answer"], inplace=True)

In [ ]:
df.isna().sum()

,0
question,0
answer,0


In [ ]:
import re

def arabic_clean(text):
    text = str(text)
    text = re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', text)
    text = re.sub(r'[ًٌٍَُِّْـ]', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

df['question'] = df['question'].apply(arabic_clean)
df['answer'] = df['answer'].apply(arabic_clean)

In [ ]:
!pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 70.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import faiss


In [ ]:
retriever = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
questions = df['question'].tolist()
embeddings = retriever.encode(questions, show_progress_bar=True)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/435 [00:00<?, ?it/s]

In [30]:
def retrieve_answer(query):

    query_clean = query.strip()


    matches = df[df['question'] == query_clean]

    if matches.empty:
        return None
    else:
        return matches.iloc[0]['answer']


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-base")
model = AutoModelForCausalLM.from_pretrained("aubmindlab/aragpt2-base")

model.safetensors:   0%|          | 0.00/553M [00:00<?, ?B/s]

In [ ]:
def generate_simplified_answer(answer, max_length=120):

    prompt = f"""\
دي إجابة قانونية رسمية:
{answer}

عاوزك تشرحلي بلغة سهلة كأني واحد مش فاهم في القانون خالص، وبالعامية المصرية لو ينفع:
"""


    inputs = tokenizer.encode(prompt.strip(), return_tensors="pt", truncation=True, max_length=512)


    outputs = model.generate(
        inputs,
        max_length=max_length,
        do_sample=True,
        top_k=40,
        top_p=0.95,
        temperature=0.2,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )


    result = tokenizer.decode(outputs[0], skip_special_tokens=True)


    if "عاوزك" in result:
        result = result.split("عاوزك")[0].strip()
    elif "بالعامية" in result:
        result = result.split("بالعامية")[0].strip()

    return result.strip()

In [ ]:
!pip install ddgs



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.0 MB/s eta 0:00:00


In [ ]:
from ddgs import DDGS

def search_web(query, max_results=3):
    try:
        with DDGS() as ddgs:
            results = ddgs.text(query, region='eg-en', safesearch='Moderate', max_results=max_results)
            return [result['body'] for result in results if 'body' in result]
    except Exception as e:
        print("❌ Error during search:", e)
        return []


In [ ]:
print(search_web("ما هو الطعن بالنقض؟"))


['طرق الطعن في المحررات العرفية ... قانون العلاقات الدبلوماسية والقنصلية علما وعملا مع إشارة خاصة لما هو مطبق في مصر / أحمد أبو الوفا. by. أبو الوفا، أحمد.', '... الطعن وميعاد جلسة نظر الطعن أمام المحكمة الإدارية العليا. Volume 10, Issue 4, December 2024, Page 2041-2066. چورچ سامي نقولا فهمي. View Article; | PDF 573.29 K ...', '“القضاء الأعلى” يعدل عودة العمل بالنقض والاستئناف والابتدائية لما بعد عيد الفطر ... 7 حالات لا يجوز الطعن عليها أمام محكمة النقض · read more. 13 April 2020 ...']


In [ ]:
print(search_web("كم عدد سنين حكم المؤبد "))


['السَّجْن المؤبد أو الحَبْس المؤبد أو الحكم المؤبد هو حكم تصدره محكمة على جريمة يقضي بموجبها الجاني ما تبقى من حياته في السجن، ويختلف مقداره من دولة لأخرى، فقد تجده في أحدها يعني الحكم لمدة 100 ...', 'Oct 7, 2019 · يذهب البعض إلى أن عقوبة السجن المؤبد هى فترة زمنية تكون ما بين 20 أو 25 سنة، إلا أن هذا الاعتقاد غير صحيح، فبحسب القانون رقم (15) لسنة 1976 .', 'كم مدة السجن المؤبد ؟ تستغرق مده السجن المؤبد حوالى 25 او 30 سنه تقريبا حسب العقوبه وحسب سلوك المسجون وتعتبر هذه المده كبيره جدا وعقاب مؤلم لمن تسول له نفسه']


In [31]:
q = "كام سنة المؤبد"
orig = retrieve_answer(q)

if orig is None:
    print("❗ السؤال غير موجود في قاعدة البيانات القانونية.")
else:
    simp = generate_simplified_answer(orig)
    print("الإجابة الأصلية:", orig)
    print("التبسيط المقترح:", simp)


❗ السؤال غير موجود في قاعدة البيانات القانونية.


In [36]:

def retrieve_answer(query):
    query_clean = query.strip()
    matches = df[df['question'] == query_clean]
    if matches.empty:
        return None
    else:
        return matches.iloc[0]['answer']


q = "على من ترفع الدعوى الجزائية الخاصة؟"
orig = retrieve_answer(q)

if orig is None:
    print("❗ السؤال غير موجود في قاعدة البيانات القانونية.")
else:
    simp = generate_simplified_answer(orig)
    print("الإجابة الأصلية:", orig)
    print("التبسيط المقترح:", simp)


الإجابة الأصلية: ترفع دعوى الحق الخاص على المتهم إذا كان أهلًا، وعلى الولي أو الوصي إذا كان المتهم ناقص الأهلية، فإن لم يكن للمتهم ولي أو وصي؛ وجب على المحكمة المرفوعة أمامها الدعوى الجزائية أن تقيم عليه وليًا.
التبسيط المقترح: دي إجابة قانونية رسمية:
ترفع دعوى الحق الخاص على المتهم إذا كان أهلًا، وعلى الولي أو الوصي إذا كان المتهم ناقص الأهلية، فإن لم يكن للمتهم ولي أو وصي؛ وجب على المحكمة المرفوعة أمامها الدعوى الجزائية أن تقيم عليه وليًا.


In [37]:
!pip install -q streamlit pyngrok faiss-cpu sentence-transformers transformers accelerate torch ddgs scikit-learn


In [51]:
%%writefile yousif.py
import streamlit as st
import pandas as pd, re, os


st.set_page_config(page_title="المساعد القانوني المبسّط", page_icon="⚖️", layout="centered")
st.markdown("<h1 style='text-align:center'>⚖️ المساعد القانوني المبسّط</h1>", unsafe_allow_html=True)
st.write("اكتب سؤالك القانوني وسنُرجع الإجابة الرسمية من قاعدة البيانات ثم تبسيطها بالعامية. إذا لم نجدها في القاعدة، نعرض ملخصات من الويب.")


CSV_PATH = "/content/drive/MyDrive/Youssif/Legal data.csv"


def arabic_clean(text):
    text = str(text)
    text = re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', text)
    text = re.sub(r'[ًٌٍَُِّْـ]', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


def load_df(path):
    if not os.path.exists(path):
        return None, "لا يمكن العثور على قاعدة البيانات في المسار المحدد."
    try:
        df = pd.read_csv(path)
        if not {"question","answer"}.issubset(df.columns):
            return None, "الملف لا يحتوي الأعمدة المطلوبة (question, answer)."
        df = df.dropna(subset=["question","answer"]).copy()
        df["question"] = df["question"].apply(arabic_clean)
        df["answer"]   = df["answer"].apply(arabic_clean)
        return df, None
    except Exception:
        return None, "تعذر قراءة الملف. تأكد من الترميز/التنسيق."

df, df_err = load_df(CSV_PATH)


def retrieve_answer(query: str):
    if df is None:
        return None
    q = arabic_clean(query)
    m = df[df["question"] == q]
    return None if m.empty else m.iloc[0]["answer"]


from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-base")
model     = AutoModelForCausalLM.from_pretrained("aubmindlab/aragpt2-base")

def generate_simplified_answer(answer, max_length=256):
    prompt = f"""\
دي إجابة قانونية رسمية:
{answer}

عاوزك تشرحلي بلغة سهلة كأني واحد مش فاهم في القانون خالص، وبالعامية المصرية لو ينفع:
"""
    inputs = tokenizer.encode(prompt.strip(), return_tensors="pt", truncation=True, max_length=512)
    outputs = model.generate(
        inputs, max_length=max_length, do_sample=True,
        top_k=40, top_p=0.95, temperature=0.2,
        num_return_sequences=1, pad_token_id=tokenizer.eos_token_id
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "عاوزك" in result:   result = result.split("عاوزك")[0].strip()
    elif "بالعامية" in result: result = result.split("بالعامية")[0].strip()
    return result.strip()


from ddgs import DDGS
def search_web(query, max_results=3):
    try:
        with DDGS() as d:
            res = d.text(query, region="eg-ar", safesearch="Moderate", max_results=max_results)
            return [r.get("body","") for r in res if r.get("body")]
    except Exception:
        return []


user_q = st.text_area("✍️ اكتب سؤالك القانوني هنا:", value="", height=120, placeholder="")

if df_err:
    st.error(df_err)

if st.button("🔎 بحث") and user_q.strip():
    if df_err:

        web = search_web(user_q, max_results=3)
        if web:
            st.subheader("🌐 ملخصات من الويب:")
            for i, w in enumerate(web, 1):
                st.markdown(f"**نتيجة {i}:** {w}")
        else:
            st.warning("لا توجد نتائج مناسبة من الويب الآن.")
    else:

        ans = retrieve_answer(user_q)
        if ans:
            st.success("✅ تم العثور على إجابة في قاعدة البيانات.")
            st.subheader("📄 الإجابة الرسمية:")
            st.write(ans)

            st.subheader("💬 الشرح المبسّط (عامية مصرية):")
            simp = generate_simplified_answer(ans, max_length=256)
            st.write(simp if simp else "—")
        else:

            st.warning("لم نجد السؤال في قاعدة البيانات. نعرض نتائج من الويب:")
            web = search_web(user_q, max_results=3)
            if web:
                st.subheader("🌐 ملخصات من الويب:")
                for i, w in enumerate(web, 1):
                    st.markdown(f"**نتيجة {i}:** {w}")
            else:
                st.info("لا توجد نتائج مناسبة من الويب الآن. حاول تغيير صياغة السؤال.")


Writing yousif.py


In [53]:
import subprocess, time
from pyngrok import ngrok


try:
    for t in ngrok.get_tunnels():
        ngrok.disconnect(t.public_url)
    ngrok.kill()
except Exception:
    pass


subprocess.run(["pkill","-f","streamlit"], check=False)
subprocess.run(["fuser","-k","8501/tcp"], check=False)

time.sleep(2)


p = subprocess.Popen(
    ["streamlit","run","yousif.py","--server.port","8501","--server.address","0.0.0.0"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)



public_url = ngrok.connect(8501, bind_tls=True).public_url
print("Public URL:", public_url)


time.sleep(5)
for _ in range(20):
    line = p.stdout.readline()
    if not line: break
    print(line.strip())


Public URL: https://5331443012cf.ngrok-free.app



You can now view your Streamlit app in your browser.

URL: http://0.0.0.0:8501

2025-08-10 00:22:28.584524: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754785348.598021   30571 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754785348.601518   30571 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754785348.614211   30571 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754785348.614247   30571 computation_placer.cc:177] computation placer already registered. Please check linkage and av